# Cells below save explanation images for a specific model - change model path/classification layer accordingly
- in the case of BlurPool or FLCPooling: need to adjust more layers

In [7]:
# LETS USE no augmentations for comparisons! sounds better and seems to have more logical results

import os
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import numpy as np
import pandas as pd
from torchvision.transforms import functional as TF
from PIL import Image
from libraries.bcosconv2d import NormedConv2d
import pydicom 
import random
import matplotlib.pyplot as plt

from collections import OrderedDict

from libraries.bcosconv2d import NormedConv2d
from libraries.bcoslinear import BcosLinear
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d


np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

# Paths
csv_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"

#csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"
#model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_BCos\no_nosamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"
model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_BCos\light_oversamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"
with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
state_dict = torch.load(model_path, map_location=device)

model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

# Transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

data = pd.read_csv(csv_path)
first_split = splits[0]
val_idx = first_split[1]  # Only use the validation indices from the 5th split
val_data = data.iloc[val_idx]
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


directory = r"C:\Users\Admin\Documents\MasterThesis\comparison_images\Blur_Results"
os.makedirs(directory, exist_ok=True)


i = 0
with torch.no_grad(), model.explanation_mode():
    for images, labels, patient_ids in val_loader:
        labels = labels.to(device)
        six_channel_images = []
        if i > 100:
            break   
        for img_tensor in images:
            numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            pil_image = Image.fromarray(numpy_image)
            transformed_image = model.transform(pil_image)
            six_channel_images.append(transformed_image)
        six_channel_images = torch.stack(six_channel_images).to(device)
        for image, patient_id in zip(six_channel_images, patient_ids):
          i += 1
          if i > 100:
            break
          image = image[None]
          expl = model.explain(image)
          filename = f"{patient_id}_normal_explanation.png"
          image_path = os.path.join(directory, filename)
          plt.figure()
          plt.imshow(expl["explanation"])
          plt.axis('off')
          plt.savefig(image_path, bbox_inches="tight", pad_inches=0)
          plt.close()

Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


In [6]:
# LETS USE no augmentations for comparisons! sounds better and seems to have more logical results

import os
import pickle
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
import numpy as np
import pandas as pd
from torchvision.transforms import functional as TF
from PIL import Image
from libraries.bcosconv2d import NormedConv2d
import pydicom 
import random
import matplotlib.pyplot as plt

from collections import OrderedDict

from libraries.bcosconv2d import NormedConv2d
from libraries.bcoslinear import BcosLinear
from pooling.flc_bcosconv2d import ModifiedFLCBcosConv2d
from pooling.blur_bcosconv2d import ModifiedBcosConv2d



np.random.seed(1)
random.seed(1)
torch.manual_seed(1)

# Paths
csv_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\grouped_data.csv"

#csv_path = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_labels.csv"
image_folder = r"C:\Users\Admin\Documents\rsna-pneumonia-detection-challenge\stage_2_train_images"
splits_path = r"G:\Meine Ablage\Universität\Master Thesis\Pneumonia\training\splits\splits_balanced_fix.pkl"
#model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_FLC\no_nosamp\seed_0\pneumonia_detection_model_resnet_bestf1_1.pth"
model_path = r"C:\Users\Admin\Documents\MasterThesis\results\Pneumonia\ResNet50_FLC_1.5B\no_nosamp\seed_0\pneumonia_detection_model_resnet_bestf1_3.pth"

with open(splits_path, 'rb') as f:
    splits = pickle.load(f)

class PneumoniaDataset(Dataset):
    def __init__(self, dataframe, image_folder, transform=None):
        self.data = dataframe
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = os.path.join(self.image_folder, f"{row['patientId']}.dcm")
        label = row['Target']
        patient_id = row['patientId']

        # Load DICOM file and process it into RGB format
        dicom = pydicom.dcmread(image_path)
        image = dicom.pixel_array
        image = Image.fromarray(image).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long), patient_id
    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('B-cos/B-cos-v2', 'resnet50', pretrained=True)

model.layer2[0].conv2 = ModifiedFLCBcosConv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=1.5, transpose=True)
model.layer2[0].downsample[0] = ModifiedFLCBcosConv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), b=1.5, transpose=False)

model.layer3[0].conv2 = ModifiedFLCBcosConv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=1.5, transpose=True)
model.layer3[0].downsample[0] = ModifiedFLCBcosConv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), b=1.5, transpose=False)

model.layer4[0].conv2 = ModifiedFLCBcosConv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), b=1.5, transpose=True)
model.layer4[0].downsample[0] = ModifiedFLCBcosConv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), b=1.5, transpose=False)    
model.fc.linear = NormedConv2d(2048, 2, kernel_size=(1, 1), stride=(1, 1), bias=False) # code from B-cos paper reused to adjust network
state_dict = torch.load(model_path, map_location=device)

model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

# Transformations
transform = transforms.Compose([
    transforms.ToTensor()
])

data = pd.read_csv(csv_path)
first_split = splits[3]
val_idx = first_split[1]  # Only use the validation indices from the 5th split
val_data = data.iloc[val_idx]
val_dataset = PneumoniaDataset(val_data, image_folder, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


directory = r"C:\Users\Admin\Documents\MasterThesis\comparison_images\FLC_LowB"
os.makedirs(directory, exist_ok=True)


i = 0
with torch.no_grad():
    for images, labels, patient_ids in val_loader:
        labels = labels.to(device)
        six_channel_images = []
        if i > 100:
            break   
        for img_tensor in images:
            numpy_image = (img_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)
            pil_image = Image.fromarray(numpy_image)
            transformed_image = model.transform(pil_image)
            six_channel_images.append(transformed_image)
        six_channel_images = torch.stack(six_channel_images).to(device)
        for image, patient_id in zip(six_channel_images, patient_ids):
          i += 1
          if i > 100:
            break
          image = image[None]
          expl = model.explain(image)
          filename = f"{patient_id}_flc_explanation_bad.png"
          image_path = os.path.join(directory, filename)

          plt.figure()
          plt.imshow(expl["explanation"])
          plt.axis('off')
          plt.savefig(image_path, bbox_inches="tight", pad_inches=0)
          plt.close()

Using cache found in C:\Users\Admin/.cache\torch\hub\B-cos_B-cos-v2_main
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\bcos\common.py:152: UserWarning: Input tensor did not require grad! Has been set automatically to True!
  warnings.warn(


Before padding x: torch.Size([1, 128, 56, 56])
After padding x: torch.Size([1, 128, 57, 57])
Before removing padding something: torch.Size([1, 128, 29, 29])
After removing padding something: torch.Size([1, 128, 28, 28])
Before padding x: torch.Size([1, 512, 56, 56])
After padding x: torch.Size([1, 512, 57, 57])
Before removing padding something: torch.Size([1, 512, 29, 29])
After removing padding something: torch.Size([1, 512, 28, 28])
Before padding x: torch.Size([1, 256, 28, 28])
After padding x: torch.Size([1, 256, 29, 29])
Before removing padding something: torch.Size([1, 256, 15, 15])
After removing padding something: torch.Size([1, 256, 14, 14])
Before padding x: torch.Size([1, 1024, 28, 28])
After padding x: torch.Size([1, 1024, 29, 29])
Before removing padding something: torch.Size([1, 1024, 15, 15])
After removing padding something: torch.Size([1, 1024, 14, 14])
Before padding x: torch.Size([1, 512, 14, 14])
After padding x: torch.Size([1, 512, 15, 15])
Before removing padding